### This example from pytorch site

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
# download training data
trainig_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:02<00:00, 8961237.84it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 214163.55it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:01<00:00, 3965761.42it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 2420387.51it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
# download testing data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [5]:
batch_size = 64

In [6]:
# create dataloader
train_dataloader = DataLoader(trainig_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [8]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models
To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU or MPS if available.

In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.liner_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.liner_relu_stack(x)
        return logits

In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### Optimizing the Model Parameters

In [16]:
# loss function
loss_fn = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Training

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # back propagation
        loss.backward()
        optimizer.step()
        # Sets the gradients of all optimized :class:`torch.Tensor` s to zero.
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:5d}]")

In [21]:
# check model performance
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \nAccuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f}\n")

In [22]:
epoch = 5
for t in range(epoch):
    print(f"Epoch: {(t+1)}\n----------------------------------")
    train(dataloader=train_dataloader, model=model, loss_fn=loss_fn, optimizer=optimizer)
    test(test_dataloader, model, loss_fn)
print("\nDone")

Epoch: 1
----------------------------------
loss: 2.189375 [   64/60000]
loss: 2.180459 [ 6464/60000]
loss: 2.122905 [12864/60000]
loss: 2.138512 [19264/60000]
loss: 2.118757 [25664/60000]
loss: 2.042106 [32064/60000]
loss: 2.077082 [38464/60000]
loss: 1.997232 [44864/60000]
loss: 2.003647 [51264/60000]
loss: 1.949589 [57664/60000]
Test error: 
Accuracy: 57.3%, Avg Loss: 1.940947

Epoch: 2
----------------------------------
loss: 1.970342 [   64/60000]
loss: 1.945970 [ 6464/60000]
loss: 1.828558 [12864/60000]
loss: 1.869662 [19264/60000]
loss: 1.793903 [25664/60000]
loss: 1.717720 [32064/60000]
loss: 1.750330 [38464/60000]
loss: 1.638441 [44864/60000]
loss: 1.665959 [51264/60000]
loss: 1.572864 [57664/60000]
Test error: 
Accuracy: 60.1%, Avg Loss: 1.579188

Epoch: 3
----------------------------------
loss: 1.644096 [   64/60000]
loss: 1.608528 [ 6464/60000]
loss: 1.451010 [12864/60000]
loss: 1.523873 [19264/60000]
loss: 1.428434 [25664/60000]
loss: 1.394665 [32064/60000]
loss: 1.422745

### Saving model

In [23]:
# torch.save(model.state_dict(), "model.pth")
# print("Saved model named model.pth")

### Prediction

In [24]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [29]:
model.eval()
# x, y = test_data[0][0], test_data[0][1]
t, i = 0, 0
for x, y in test_data:
    with torch.no_grad():
        x = x.to(device)
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        if predicted == actual:
            t += 1
        print(f"Predicted: {predicted}, Actual: {actual}")
    i += 1
    
print(f"{(t/i)*100:.2f}% accuracy")

Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Trouser, Actual: Trouser
Predicted: Trouser, Actual: Trouser
Predicted: Shirt, Actual: Shirt
Predicted: Trouser, Actual: Trouser
Predicted: Coat, Actual: Coat
Predicted: Pullover, Actual: Shirt
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: Pullover, Actual: Coat
Predicted: Ankle boot, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Coat, Actual: Coat
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Pullover
Predicted: Pullover, Actual: Coat
Predicted: Bag, Actual: Bag
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Shirt, Actual: Pullover
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: Sneaker, Actual: Ankle boot
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Coat
Predicted: Coat, Actual: Shirt
Predicted: Dress, Actual: T-shirt/top
Predicted:

Predicted: Sneaker, Actual: Ankle boot
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Shirt
Predicted: Pullover, Actual: Coat
Predicted: Bag, Actual: Bag
Predicted: Pullover, Actual: Pullover
Predicted: T-shirt/top, Actual: Shirt
Predicted: Sandal, Actual: Sandal
Predicted: Pullover, Actual: Coat
Predicted: Coat, Actual: Shirt
Predicted: Sneaker, Actual: Sneaker
Predicted: Sneaker, Actual: Sneaker
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Dress, Actual: Dress
Predicted: Sneaker, Actual: Sneaker
Predicted: Coat, Actual: T-shirt/top
Predicted: Sneaker, Actual: Sneaker
Predicted: Trouser, Actual: Trouser
Predicted: Dress, Actual: Dress
Predicted: Sneaker, Actual: Sneaker
Predicted: Pullover, Actual: Pullover
Predicted: Pullover, Actual: Pullover
Predicted: Dress, Actual: Dress
Predicted: Coat, Actual: Coat
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Dress, Actual: Dress
Predicted: Trouser, Actual: Trouser
Predicted: T-shirt

Predicted: Dress, Actual: Dress
Predicted: Coat, Actual: Shirt
Predicted: Bag, Actual: Bag
Predicted: Coat, Actual: Coat
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Bag, Actual: Bag
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Dress, Actual: Coat
Predicted: Sneaker, Actual: Sandal
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Coat
Predicted: Coat, Actual: Coat
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Trouser, Actual: Trouser
Predicted: Sneaker, Actual: Sneaker
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Sneaker, Actual: Sandal
Predicted: Bag, Actual: Bag
Predicted: Dress, Actual: Trouser
Predicted: Ankle boot, Actual: Sneaker
Predicted: Sneaker, Actual: Sneaker
Predicted: Trouser, Actual: Trouser
Predicted: Trouser, Actual: Trouser
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Shirt
Predicted:

Predicted: Coat, Actual: Coat
Predicted: Pullover, Actual: Pullover
Predicted: Coat, Actual: Shirt
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sandal
Predicted: T-shirt/top, Actual: Shirt
Predicted: Dress, Actual: Coat
Predicted: Sneaker, Actual: Sandal
Predicted: Ankle boot, Actual: Sandal
Predicted: Bag, Actual: Bag
Predicted: Trouser, Actual: Trouser
Predicted: Dress, Actual: Trouser
Predicted: Sneaker, Actual: Sneaker
Predicted: Coat, Actual: Coat
Predicted: T-shirt/top, Actual: Shirt
Predicted: Ankle boot, Actual: Ankle boot
Predicted: T-shirt/top, Actual: Shirt
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sneaker
Predicted: Coat, Actual: Pullover
Predicted: Pullover, Actual: Pullover
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Pullover
Predicted: Sneaker, Actual: Sneaker
Predicted: Sneaker, Actual: Sneaker
Predicted: Pullover, Actual: Shirt
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Bag, Actual: Bag
Predicted: Ankle boo

Predicted: Dress, Actual: Dress
Predicted: Dress, Actual: Dress
Predicted: Trouser, Actual: Trouser
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Bag
Predicted: Sneaker, Actual: Sandal
Predicted: Dress, Actual: Dress
Predicted: Bag, Actual: Bag
Predicted: T-shirt/top, Actual: Shirt
Predicted: T-shirt/top, Actual: Dress
Predicted: Pullover, Actual: Coat
Predicted: Dress, Actual: Dress
Predicted: Pullover, Actual: Pullover
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Pullover, Actual: Pullover
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Bag, Actual: Shirt
Predicted: Bag, Actual: Bag
Predicted: Dress, Actual: Dress
Predicted: Sneaker, Actual: Ankle boot
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Trouser, Actual: Trouser
Predicted: Sneaker, Actual: Sneaker
Predicted: Ankle boot, Actual: Sandal
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Coat, Actual: Pullover
Predicted: Dress, 

...

Predicted: Sneaker, Actual: Sneaker
Predicted: Bag, Actual: Shirt
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: T-shirt/top, Actual: Shirt
Predicted: Shirt, Actual: Coat
Predicted: Sneaker, Actual: Sandal
Predicted: Bag, Actual: Trouser
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sneaker
Predicted: Coat, Actual: Coat
Predicted: T-shirt/top, Actual: Shirt
Predicted: Ankle boot, Actual: Sandal
Predicted: Trouser, Actual: Trouser
Predicted: Ankle boot, Actual: Sandal
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Shirt
Predicted: Dress, Actual: T-shirt/top
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Ankle boot, Actual: Sandal
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Coat, Actual: Ankle boot
Predicted: Bag, Actual: Bag
Predicted: Dress, Actual: Dress
Predicte

Predicted: Sneaker, Actual: Sneaker
Predicted: Sneaker, Actual: Sneaker
Predicted: Trouser, Actual: Trouser
Predicted: Sneaker, Actual: Ankle boot
Predicted: Sandal, Actual: Sandal
Predicted: Dress, Actual: Coat
Predicted: T-shirt/top, Actual: Trouser
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Sneaker, Actual: Sneaker
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Dress
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Shirt
Predicted: Bag, Actual: Pullover
Predicted: Sandal, Actual: Sandal
Predicted: Ankle boot, Actual: Sandal
Predicted: Coat, Actual: Coat
Predicted: Pullover, Actual: Shirt
Predicted: Sneaker, Actual: Sneaker
Predicted: T-shirt/top, Actual: Pullover
Predicted: T-shirt/top, Actual: Shirt
Predicted: Dress, Actual: Dress
Predicted: Coat, Actual: Pullover
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Coat, Actual: Coat
Predicted: Sneaker, Actual: Sneaker
Predicted: Bag, Actual: Bag
Predicted: Pull

Predicted: Ankle boot, Actual: Ankle boot
Predicted: Coat, Actual: Coat
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sneaker
Predicted: Pullover, Actual: Shirt
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Coat, Actual: Dress
Predicted: Coat, Actual: Coat
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Coat, Actual: Coat
Predicted: Bag, Actual: Bag
Predicted: Dress, Actual: T-shirt/top
Predicted: Trouser, Actual: Trouser
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Trouser, Actual: Trouser
Predicted: Bag, Actual: Shirt
Predicted: Sneaker, Actual: Sneaker
Predicted: Dress, Actual: Trouser
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sandal
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Coat, Actual: Coat
Predicted: Coat, Actual: Shirt
Predicted: Coat, Actual: Coat
Predicted: Trouser, Actual: Trouser
Predicted: Trouser, Actual: Trouser
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Bag, Actual

Predicted: Ankle boot, Actual: Sandal
Predicted: Dress, Actual: Dress
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Pullover, Actual: Pullover
Predicted: Bag, Actual: Bag
Predicted: Sandal, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Sneaker, Actual: Sneaker
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Coat, Actual: Shirt
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Pullover
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Coat
Predicted: Bag, Actual: Bag
Predicted: Sneaker, Actual: Sandal
Predicted: Dress, Actual: Pullover
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Trouser, Actual: Trouser
Predicted: Pullover, Actual: Shirt
Predicted: Coat, Actual: Shirt
Predicted: Pullover, Actual: Trouser
Predicted: Sneaker, Actual: Sneaker
Predicted: Coat, Actual: Shirt
Predicted: Trouser, Actual: Trouser
P

Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: T-shirt/top, Actual: Dress
Predicted: Dress, Actual: Dress
Predicted: T-shirt/top, Actual: Shirt
Predicted: Dress, Actual: Dress
Predicted: Shirt, Actual: T-shirt/top
Predicted: Bag, Actual: Bag
Predicted: Coat, Actual: Shirt
Predicted: Sneaker, Actual: Sneaker
Predicted: Ankle boot, Actual: Sandal
Predicted: Sneaker, Actual: Sandal
Predicted: Sneaker, Actual: Sneaker
Predicted: T-shirt/top, Actual: T-shirt/top
Predicted: Sneaker, Actual: Sneaker
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Coat
Predicted: Dress, Actual: Coat
Predicted: Pullover, Actual: Pullover
Predicted: Ankle boot, Actual: Ankle boot
Predicted: Pullover, Actual: Trouser
Predicted: Ankle boot, Actual: Sneaker
Predicted: Trouser, Actual: Trouser
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Bag, Actual: Bag
Predicted: Ankle boot, Actual: Sandal
Predicted: Bag, Actual: Bag
Predicted: Dress, Actual: Dress
Predicted: Ba